In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

X_VER = 186

L_VER = 514
USE = 'clicks'

# USE 0,1,2,3
PART = 0

In [2]:
import pandas as pd, cudf, numpy as np, gc
import matplotlib.pyplot as plt

In [3]:
users = pd.read_parquet(f'../../data/infer_with_features/train_v161_LB_{PART}.pqt')

# XGBoost

In [4]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

152
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [5]:
%%time
print('BEFORE', users.shape )
users = users.sort_values('user').reset_index(drop=True)
print('AFTER', users.shape )

BEFORE (41795000, 154)
AFTER (41795000, 154)
CPU times: user 12.6 s, sys: 9.99 s, total: 22.6 s
Wall time: 19.3 s


In [6]:
oof = np.zeros(len(users))

In [28]:
%%time
from cuml import ForestInference
FOLDS = 5

for fold in range(FOLDS):
    print('#'*25)
    print('### Fold',fold+1)
    print('#'*25)
    
    print(f'=> predicting XGB {X_VER}...')
    #model = xgb.Booster()
    #model.load_model(f'../../data/models/XGB_fold{fold}_{USE}_v{X_VER}.xgb')
    #model.set_param({'predictor': 'gpu_predictor'})
    #model.set_param({'gpu_id': 0})
    model = ForestInference.load(f'../../data/models/XGB_fold{fold}_{USE}_v{X_VER}.xgb')
    
    # INFER OOF FOLD K
    print('inferring 1of4...')
    X_valid = users[FEATURES].iloc[:len(users)//4]
    #dvalid = xgb.DMatrix(data=X_valid) 
    oof_preds = model.predict(X_valid)
    oof[:len(users)//4] += oof_preds/FOLDS
    del X_valid#, dvalid
    gc.collect()
    
    # INFER OOF FOLD K
    print('inferring 2of4...')
    X_valid = users[FEATURES].iloc[len(users)//4:(2*len(users))//4]
    #dvalid = xgb.DMatrix(data=X_valid) 
    oof_preds = model.predict(X_valid)
    oof[len(users)//4:(2*len(users))//4] += oof_preds/FOLDS
    del X_valid#, dvalid
    gc.collect()
    
    # INFER OOF FOLD K
    print('inferring 3of4...')
    X_valid = users[FEATURES].iloc[(2*len(users))//4:(3*len(users))//4]
    #dvalid = xgb.DMatrix(data=X_valid) 
    oof_preds = model.predict(X_valid)
    oof[(2*len(users))//4:(3*len(users))//4] += oof_preds/FOLDS
    del X_valid#, dvalid
    gc.collect()
    
    print('inferring 4of4...')
    X_valid = users[FEATURES].iloc[(3*len(users))//4:]
    #dvalid = xgb.DMatrix(data=X_valid) 
    oof_preds = model.predict(X_valid)
    oof[(3*len(users))//4:] += oof_preds/FOLDS

    del X_valid, model#, dvalid 
    gc.collect()

#########################
### Fold 1
#########################
=> predicting XGB 186...
inferring first half...
inferring second half...
#########################
### Fold 2
#########################
=> predicting XGB 186...
inferring first half...
inferring second half...
#########################
### Fold 3
#########################
=> predicting XGB 186...
inferring first half...
inferring second half...
#########################
### Fold 4
#########################
=> predicting XGB 186...
inferring first half...
inferring second half...
CPU times: user 1min 58s, sys: 1min 1s, total: 2min 59s
Wall time: 2min 58s


In [29]:
%%time
if 0:
    for c in users.columns[2:]:
        print(c,', ',end='')
        if str(users[c].dtype)=='float32':
            users[c] = users[c].astype('float16')
        if str(users[c].dtype)=='int32':
            users[c] = users[c].astype('int16')

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 14.8 µs


In [30]:
%%time
oof2 = np.zeros(len(users))

if 0:
    for fold in range(5):
        print('#'*25)
        print('### Fold',fold+1)
        print('#'*25)

        print(f'=> predicting LGB {L_VER}...')
        model = lgb.Booster( model_file=f'../../jan-22-23-otto/ranker_models/LGB_fold{fold}_{USE}_v{L_VER}.lgb' )
        oof_preds = model.predict(users[FEATURES2])
        oof2 += oof_preds/FOLDS

        del model
        gc.collect()

CPU times: user 227 µs, sys: 95 µs, total: 322 µs
Wall time: 273 µs


# Load Preds

In [31]:
if 0:
    oof = np.load(f'/raid/Kaggle/otto/pred10/pred{X_VER}_v{PART}.npy')
    oof2 = np.load(f'/raid/Kaggle/otto/pred10/pred{L_VER}_v{PART}.npy')

# Make Sub

In [32]:
sub = users[['user','item']].copy()

sub['p1'] = oof
#sub['p2'] = oof2

print( sub.shape )
sub.head()

(41795000, 3)


,user,item,p1
0,12899779,59625,0.828945
1,12899779,1400007,0.009453
2,12899779,1628974,0.018735
3,12899779,1104389,0.009164
4,12899779,642225,0.023598


In [33]:
%%time
if 0:
    sub['p1'] = sub.groupby('user').p1.rank()
    #sub['p2'] = sub.groupby('user').p2.rank()

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 15.7 µs


In [34]:
%%time
sub['p'] = sub.p1

sub = sub.sort_values(['user','p'],ascending=[True,False])
print( sub.shape )
sub.head()

(41795000, 4)
CPU times: user 41.6 s, sys: 3 s, total: 44.6 s
Wall time: 44.6 s


,user,item,p1,p
0,12899779,59625,0.828945,0.828945
76,12899779,737445,0.535822,0.535822
71,12899779,731692,0.248380,0.248380
72,12899779,1253524,0.175566,0.175566
67,12899779,1790770,0.168614,0.168614


In [35]:
sub = sub.reset_index(drop=True)

In [36]:
sub['n'] = sub.groupby('user').item.cumcount().astype('int8')

In [37]:
sub.head()

,user,item,p1,p,n
0,12899779,59625,0.828945,0.828945,0
1,12899779,737445,0.535822,0.535822,1
2,12899779,731692,0.248380,0.248380,2
3,12899779,1253524,0.175566,0.175566,3
4,12899779,1790770,0.168614,0.168614,4


In [38]:
sub = sub.loc[sub.n<20]
print( sub.shape )

(8359000, 5)


In [39]:
%%time
sub2 = sub.groupby('user').item.apply(list)

CPU times: user 7.12 s, sys: 193 ms, total: 7.31 s
Wall time: 7.3 s


In [40]:
sub2 = sub2.to_frame().reset_index()

In [41]:
%%time
sub2.item = sub2.item.apply(lambda x: " ".join(map(str,x)))

CPU times: user 1.59 s, sys: 64.8 ms, total: 1.65 s
Wall time: 1.65 s


In [42]:
sub2.columns = ['session_type','labels']

In [43]:
sub2.session_type = sub2.session_type.astype('str')+ f'_{USE}'

In [44]:
sub2.head()

,session_type,labels
0,12899779_clicks,59625 737445 731692 1253524 1790770 475447 134...
1,12899780_clicks,1142000 736515 582732 1502122 889686 487136 97...
2,12899781_clicks,199008 918667 1836671 528496 1681537 1422724 1...
3,12899782_clicks,1007613 834354 595994 1669402 740494 1033148 3...
4,12899783_clicks,1817895 408787 1476106 1811433 58861 573058 60...


In [45]:
sub2.to_csv(f'../../data/submission_parts/submission_{USE}_v{PART}.csv',index=False)

In [46]:
#np.save(f'/raid/Kaggle/otto/pred15/pred{X_VER}_v{PART}',oof)
#np.save(f'/raid/Kaggle/otto/pred15/pred{L_VER}_v{PART}',oof2)

In [47]:
sub2.head()

,session_type,labels
0,12899779_clicks,59625 737445 731692 1253524 1790770 475447 134...
1,12899780_clicks,1142000 736515 582732 1502122 889686 487136 97...
2,12899781_clicks,199008 918667 1836671 528496 1681537 1422724 1...
3,12899782_clicks,1007613 834354 595994 1669402 740494 1033148 3...
4,12899783_clicks,1817895 408787 1476106 1811433 58861 573058 60...
